In [2]:
import pennylane as qml
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Define the quantum device with 2 qubits
dev = qml.device("default.qubit", wires=2)

# Quantum circuit architecture
def circuit(x1, x2):
    """Apply feature encoding and entangling operations."""
    qml.RY(x1, wires=0)
    qml.RY(x2, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(x1, wires=0)
    qml.RY(x2, wires=1)
    qml.CNOT(wires=[0, 1])

# Define the quantum kernel function
@qml.qnode(dev)
def quantum_kernel(x1, x2):
    circuit(x1, x2)
    return qml.expval(qml.PauliZ(0))

def kernel_matrix(X):
    """Compute the quantum kernel matrix for the dataset."""
    num_samples = len(X)
    K = np.zeros((num_samples, num_samples))
    for i in range(num_samples):
        for j in range(num_samples):
            K[i, j] = quantum_kernel(X[i, 0], X[j, 0])
    return K

# Generate simple training dataset
X_train = np.array([[-1, -1], [-1, 1], [1, -1], [1, 1]])
y_train = np.array([1, 1, -1, -1])

# Scale features to range [-1, 1] for better generalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Compute the quantum kernel matrix
K_train = kernel_matrix(X_train_scaled)

# Train an SVM classifier with the quantum kernel
svm = SVC(kernel="precomputed")  # Use precomputed quantum kernel
svm.fit(K_train, y_train)

# Test on a new data point
X_test = np.array([[0.5, 0.5]])  # New sample to classify
X_test_scaled = scaler.transform(X_test)  # Scale test data

# Compute the test kernel matrix
K_test = np.array([[quantum_kernel(X_test_scaled[0, 0], x[0]) for x in X_train_scaled]])

# Predict class label for new sample
prediction = svm.predict(K_test)
print("Prediction for new data point:", prediction)


Prediction for new data point: [1]
